# Numerical Relative Entropy Rate

In [20]:
import numpy as np
from scipy import linalg
import control as ct

In [36]:
def print_matrix(matrix, name):
    """
    Print a matrix in a readable format.
    
    Parameters:
    matrix (np.ndarray): The matrix to print.
    """
    print(f"{name}:")
    rows, cols = matrix.shape
    for i in range(rows):
        for j in range(cols):
            print(f"{matrix[i, j]:>10.4f}", end=" ")
        print()

In [21]:
def compute_powers(matrix, n):
    # Initialize the array to hold the powers of the matrix
    powers = np.empty((n, *matrix.shape), dtype=matrix.dtype)
    
    # Compute each power of the matrix
    current_power = np.eye(matrix.shape[0], dtype=matrix.dtype)  # Start with the identity matrix
    for i in range(n):
        powers[i] = current_power
        current_power = np.dot(current_power, matrix)
    
    return powers

In [53]:
nx = 10
ny = 5

context = 250

In [23]:
noise_std = 1e-1
W = np.eye(nx) * (noise_std) ** 2
V = np.eye(ny) * (noise_std) ** 2
Pi = ct.dlyap(A, W)

In [24]:
A = np.sqrt(0.33)*np.random.randn(nx, nx) #same second moment as uniform(-1,1)
A /= np.max(np.abs(linalg.eigvals(A)))
A = A * 0.95 #scale the matrix

In [25]:
Ap = np.diag(np.random.uniform(-1, 1, nx)) * 0.95
Ap[np.triu_indices(nx, 1)] = np.random.uniform(-1, 1, (nx ** 2 + nx) // 2 - nx)

In [26]:
C = np.random.normal(0, np.sqrt(0.333333333), (ny, nx))

### Compute the powers of A

In [54]:
A_powers = compute_powers(A,context)

print("shape of A_powers", A_powers.shape)

shape of A_powers (250, 10, 10)


In [56]:
#multiply by C and Pi
vals = C@A_powers@Pi@C.T
print(vals.shape)

(250, 5, 5)


In [57]:
def block_toeplitz(blocks):
    """
    Create a block Toeplitz matrix from a list of blocks.
    
    Parameters:
    blocks (list of np.ndarray): List of 2D arrays representing the blocks.
    
    Returns:
    np.ndarray: Block Toeplitz matrix.
    """
    # Determine the size of each block and the number of blocks
    block_shape = blocks[0].shape
    num_blocks = len(blocks)

    print("num_blocks", num_blocks)
    print("block_shape", block_shape)
    
    # Initialize the block Toeplitz matrix
    toeplitz_matrix = np.zeros((block_shape[0] * num_blocks, block_shape[1] * num_blocks))
    
    # Fill in the block Toeplitz matrix
    for i in range(num_blocks):
        for j in range(num_blocks):
            if i >= j:
                toeplitz_matrix[i*block_shape[0]:(i+1)*block_shape[0], j*block_shape[1]:(j+1)*block_shape[1]] = blocks[i-j].T
            else:
                toeplitz_matrix[i*block_shape[0]:(i+1)*block_shape[0], j*block_shape[1]:(j+1)*block_shape[1]] = blocks[j-i]
    
    return toeplitz_matrix

def create_block_diagonal(mat, num_blocks):
    """
    Create a block diagonal matrix with matrix mat along the diagonal and zeros elsewhere.
    
    Parameters:
    mat (np.ndarray): The matrix to place along the diagonal.
    num_blocks (int): The number of times to place mat along the diagonal.
    
    Returns:
    np.ndarray: The block diagonal matrix.
    """
    # Determine the shape of the block matrix
    block_shape = mat.shape
    block_size = block_shape[0] * num_blocks
    
    # Initialize the block diagonal matrix with zeros
    block_diagonal_matrix = np.zeros((block_size, block_size))
    
    # Fill in the block diagonal matrix with mat along the diagonal
    for i in range(num_blocks):
        start_index = i * block_shape[0]
        end_index = start_index + block_shape[0]
        block_diagonal_matrix[start_index:end_index, start_index:end_index] = mat
    
    return block_diagonal_matrix

### Create the Large Covariance Matrix

We know that Kn contains all the other iterations of Kn in its bottom right block.

In [ ]:
Kn = block_toeplitz(vals) + create_block_diagonal(V, context)

num_blocks 250
block_shape (5, 5)
(1250, 1250)


### Compute Inverses of K

https://en.wikipedia.org/wiki/Block_matrix#Inversion
$$
  \begin{bmatrix}
    {A} & {B} \\
    {C} & {D}
  \end{bmatrix}^{-1} = \begin{bmatrix}
    \left({A} - {B} {D}^{-1} {C}\right)^{-1} & {0} \\
    {0} & \left({D} - {C} {A}^{-1} {B}\right)^{-1}
  \end{bmatrix} \begin{bmatrix}
                     {I} & -{B} {D}^{-1} \\
    -{C} {A}^{-1} &                  {I}
  \end{bmatrix}.
  $$
  

  $A^{-1}$ never changes and $D^{-1}$ we computed on the last iteration.
  <span style="color: blue;">Computing the inverse using the above decomposition is slower and less accurate than scipy.linalg.inverse() on the full matrices.</span>

In [ ]:
Kinvs = []
Finvs = []
for i in range(context):
    Ki = Kn[-ny*(i+1):,-ny*(i+1):]
    Kinv = linalg.inv(Ki)
    Kinvs.append(Kinv)
    Finv = linalg.inv(Kinv[:ny, :ny])
    Finvs.append(Finv)
    print_matrix(Finv, "Finv")
    print_matrix(Kn[:ny, :ny], "K0")

In [72]:
print(len(Kinvs))

250
